<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice Using APIs

_Authors: Dave Yerrington (SF), Sam Stack(DC)_

---

In this lab we will practice using some popular APIs to retrieve and store data.

In [3]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt

%matplotlib inline

## Exercise 1: get data from Sheetsu

---

[Sheetsu](https://sheetsu.com/) is an online service that allows you to access any Google spreadsheet from an API. This can be a very powerful way to share a dataset with colleagues as well as to create a mini centralized data storage that is simpler to edit than a database.

A Google Spreadsheet with Wine data can be found [here](https://docs.google.com/spreadsheets/d/1mZ3Otr5AV4v8WwvLOAvWf3VLxDa-IeJ1AVTEuavJqeo/).

It can be accessed through sheetsu API at this endpoint: https://sheetsu.com/apis/v1.0/dab55afd

**Questions:**

1. Use the requests library to access the document. Inspect the response text. What kind of data is it?
- Check the status code of the response object. What code is it?
- Use the appropriate libraries and read functions to read the response into a Pandas Dataframe.
- Once you've imported the data into a dataframe, check the value of the 5th line: what's the price?

In [6]:
# You can either post or get info from this API
#api_base_url = 'https://sheetsu.com/apis/v1.0/dab55afd'
api_base_url ='https://sheetsu.com/apis/v1.0su/34224c7a9910'

In [7]:
# What kind of data is this returning?
api_response = requests.get(api_base_url)
api_response.text[:100]

'[{"Color":"W","Region":"Portugal","Country":"Portugal","Vintage":"2013","Vinyard":"Vinho Verde","Nam'

In [4]:
# 1. It is a json string

In [8]:
reponse = json.loads(api_response.text)

In [10]:
type(reponse)

list

In [11]:
reponse[0]

{'Color': 'W',
 'Region': 'Portugal',
 'Country': 'Portugal',
 'Vintage': '2013',
 'Vinyard': 'Vinho Verde',
 'Name': '',
 'Grape': '',
 'Consumed In': '2015',
 'Score': '4',
 'Price': ''}

In [12]:
api_response.status_code

200

In [9]:
# 2. response code is 200

In [13]:
wine_df = pd.DataFrame(reponse)
wine_df.head()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,,,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,,3,2012,Heartland


In [14]:
# alternatively:
wine_df = pd.read_json(api_response.text)
wine_df.head(2)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4.0,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3.0,2013,Peyruchet


In [15]:
wine_df.iloc[4, :]
# the price is 6 for the 5th row.

Color                     R
Consumed In            2015
Country                  US
Grape           chiraz, cab
Name           Spice Trader
Price                     6
Region                     
Score                     3
Vintage                2012
Vinyard           Heartland
Name: 4, dtype: object

## Exercise 2: post data to Sheetsu

---

We've learned how to read data, but it'd be great if we could also write data. For this we will need to use a _POST_ request.

1. Use the post command to add the `post_data` to the spreadsheet.
- What status did you get? How can you check that you actually added the data correctly?
- In this exercise, your classmates are adding data to the same spreadsheet. What happens because of this? Is it a problem? How could you mitigate it?

In [16]:
post_data = {
    'Grape': '',
    'Name': 'My wonderful wine',
    'Color': 'R',
    'Country': 'US',
    'Region': 'Sonoma',
    'Vinyard': '',
    'Score': '10',
    'Consumed In': '2015',
    'Vintage': '1973',
    'Price': '200'
}

In [17]:
requests.post(api_base_url, data=post_data)

<Response [402]>

In [15]:
# 2. To check we could send a get request and check the last line.

In [16]:
# 3. There will be many duplicate lines on the spreadsheet. One way to mitigate 
# this would be through permission, another would be to insert at a 
# specific position, so that the line is overwritten at each time.

## Exercise 3: IMDB TV Shows

---

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.

Here we will use a combination of scraping and API calls to find the ratings and networks of famous television shows.

### 3.A Get the top TV Shows

The Internet Movie Database contains data about movies and TV shows. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/toptv/?ref_=nv_tp_tv250_2 contains the list of the top 250 tv shows of all time. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

> **Hint:** movie_ids look like this: `tt2582802`
> _Everything after "/title/" and before "/?"_

In [18]:
def get_top_250():
    response = requests.get('http://www.imdb.com/chart/toptv/?ref_=nv_tp_tv250_2')
    html = response.text
    # use greedy version to find everything after title to the next backslash in the a href element
    entries = re.findall("<a href.*?/title/(.*?)/", html) 
    # create a list of the top 250 results
    return list(set(entries))

In [19]:
entries = get_top_250()

In [20]:
len(entries)

251

In [24]:
entries[0:2]

['tt1439629', 'tt1513168']

### 3.B Get data on the top movies

Although the Internet Movie Database does not have a public API, an open API exists at http://www.tvmaze.com/api.

Use this API to retrieve information about each of the 250 TV shows you have extracted in the previous step.
1. Check the documentation of tvmaze's api to learn how to request show data by id.
- Define a function that returns a python object with select information for a given id.
    - Show name
    - Rating (avg)
    - Genre(s)
    - Network name
    - Premiere date
    - Status
> Tip: the json object can easily be converted into a python dictionary.

- Store the gathered information in a Pandas Dataframe.


As the target information is in json format you will need `json.loads(res.text)` in order to gather it.

Heres and example of the information and how we can interact with it.

In [27]:
# example url
res = requests.get('http://api.tvmaze.com/lookup/shows?imdb=tt5491994')

# status code
print(res.status_code)

# just the contents of the name element
print(json.loads(res.text).get('name'))

# entire contents
print(json.loads(res.text))

200
Planet Earth II
{'id': 22036, 'url': 'http://www.tvmaze.com/shows/22036/planet-earth-ii', 'name': 'Planet Earth II', 'type': 'Documentary', 'language': 'English', 'genres': ['Nature'], 'status': 'Ended', 'runtime': 60, 'premiered': '2016-11-06', 'officialSite': 'http://www.bbc.co.uk/programmes/p02544td', 'schedule': {'time': '20:00', 'days': ['Sunday']}, 'rating': {'average': 9.7}, 'weight': 81, 'network': {'id': 12, 'name': 'BBC One', 'country': {'name': 'United Kingdom', 'code': 'GB', 'timezone': 'Europe/London'}}, 'webChannel': None, 'externals': {'tvrage': None, 'thetvdb': 318408, 'imdb': 'tt5491994'}, 'image': {'medium': 'http://static.tvmaze.com/uploads/images/medium_portrait/87/219457.jpg', 'original': 'http://static.tvmaze.com/uploads/images/original_untouched/87/219457.jpg'}, 'summary': '<p>David Attenborough presents a documentary series exploring how animals meet the challenges of surviving in the most iconic habitats on earth.</p>', 'updated': 1563541465, '_links': {'se

In [28]:
#  function to pull information from API using Json interaction
def get_entry(entry):
    res=requests.get('http://api.tvmaze.com/lookup/shows?imdb='+entry)
    if res.status_code == 200:
        try:
            status = json.loads(res.text).get('status')
        except AttributeError:
            status = 'NA'
        try: 
            rating = json.loads(res.text).get('rating').get('average')
        except AttributeError:
            rating = 'NA'
            
        try:
            network = json.loads(res.text).get('network').get('name')
        except AttributeError:
            network = 'NA'
            
        try:
            title = json.loads(res.text).get('name')
        except AttributeError:
            title = 'NA'
            
        try:
            premier = json.loads(res.text).get('premiered')
        except AttributeError:
            premier = 'NA'
            
        try:
            genres = json.loads(res.text).get('genres')
        except AttributeError:
            genres = 'NA'

        # takes local variables as a 
        shows_df.loc[len(shows_df)] = [title, rating, genres, network, premier, status]

In [29]:
#  function to pull information from API converting Json into a python dictionary element
def get_entry(entry):
    res=requests.get('http://api.tvmaze.com/lookup/shows?imdb='+entry)
    if res.status_code == 200:
        results = json.loads(res.text)
        
        try:    
            status = results['status']
        except TypeError:
            status = 'NA'   
        try:
            rating = results['rating']['average']
        except TypeError:
            rating = 'NA'
        try:
            network = results['network']['name']
        except TypeError:
            network = 'NA'
        try:   
            title = results['name']
        except TypeError:
            title = 'NA'
        try:   
            genres = results['genres']
        except TypeError:
            genres = 'NA'
        try:   
            premier = results['premiered']
        except TypeError:
            premier = 'NA'
        
        shows_df.loc[len(shows_df)] = [title, rating, genres, network, premier, status]

In [ ]:
# in both functions we are looking for specific elements.  If an element is missing an error will return thus the need
# for try and except statements.

In [30]:
shows_df= pd.DataFrame( columns = ['show_name', 'rating_avg', 'genres', 'network', 'premiere_date', 'status'])

for entry in entries:
    get_entry(entry)

In [31]:
shows_df.head()

,show_name,rating_avg,genres,network,premiere_date,status
0,Community,8.2,[Comedy],NA,2009-09-17,Ended
1,Through the Wormhole,8.3,[Science-Fiction],Science,2010-06-09,Ended
2,Feud,7.6,[Drama],FX,2017-03-05,Ended
3,Doctor Who,8.6,"[Drama, Adventure, Science-Fiction]",BBC One,2005-03-26,Running
4,Spartacus,8.7,"[Action, Adventure, History]",Starz,2010-01-22,Ended


In [32]:
shows_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231 entries, 0 to 230
Data columns (total 6 columns):
show_name        231 non-null object
rating_avg       197 non-null float64
genres           231 non-null object
network          231 non-null object
premiere_date    231 non-null object
status           231 non-null object
dtypes: float64(1), object(5)
memory usage: 12.6+ KB
